In [ ]:

# Below we will find different types of neural network architectures that can be used to solve various problems across 
# different domains, including image processing, sequence modeling, generative modeling, and various other use cases.
# Each of these neural network architectures has its strengths and weaknesses and each is suitable for different types 
# of tasks and data. Understanding the characteristics of each architecture is crucial for selecting the appropriate 
# model for a given problem.


In [ ]:

#######################################################################################################
# Feedforward Neural Network (FNN):

# A Feedforward Neural Network (FNN), also known as a Multilayer Perceptron (MLP), is a type of artificial neural 
# network where connections between the nodes do not form cycles. In other words, the information flows in one direction, 
# from the input layer through one or more hidden layers to the output layer, without any loops or feedback connections.
# So, essentially, information flows in one direction (from input to output).

# Feedforward Neural Networks are commonly used for various supervised learning tasks, including:

# Classification: FNNs can be used for classification tasks where the goal is to assign input data points to one of 
# several predefined categories. For example, classifying emails as spam or not spam, classifying images of digits 
# into their respective categories (0-9), or categorizing news articles into different topics.

# Regression: FNNs can also be used for regression tasks where the goal is to predict a continuous numerical value 
# based on input data. For example, predicting house prices based on features such as location, size, and number of 
# bedrooms, or predicting the sales volume of a product based on various marketing factors.

# Pattern Recognition: FNNs are effective for recognizing patterns in input data, making them suitable for tasks such as
# handwriting recognition, speech recognition, and facial recognition.

# Function Approximation: FNNs can approximate complex mathematical functions, making them useful for tasks such as 
# function approximation, curve fitting, and modeling complex relationships between input and output variables.


# In the example below, we will focus on handwriting recognition, specifically handwritten numbers 1-9

import numpy as np
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import mnist
import cv2


# Load the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Preprocess the data
X_train = X_train.reshape((X_train.shape[0], -1)).astype('float32') / 255.0
X_test = X_test.reshape((X_test.shape[0], -1)).astype('float32') / 255.0

# Convert the labels to one-hot encoding
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the model architecture
model = Sequential([
    Dense(512, activation='relu', input_shape=(784,)),
    Dense(256, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))



# Make predictions on test data
predictions = model.predict(X_test)

# Evaluate the predictions
predicted_labels = np.argmax(predictions, axis=1)

# Print some example predictions
for i in range(10):
    print("Predicted:", predicted_labels[i])



# Load the image containing the handwritten digit (the image is the number 7)
image = cv2.imread('C:\\Users\\ryan_\\Desktop\\number_image.jpg', cv2.IMREAD_GRAYSCALE)

# Resize the image to 28x28 pixels (same as MNIST images)
image_resized = cv2.resize(image, (28, 28))

# Invert the image (if necessary, depending on how the model was trained)
image_inverted = cv2.bitwise_not(image_resized)

# Flatten the image and normalize pixel values
input_data = image_inverted.reshape((1, -1)).astype('float32') / 255.0

# Make prediction
prediction = model.predict(input_data)

# Get the predicted label
predicted_label = np.argmax(prediction)

print("Predicted label:", predicted_label)

# Final Result: 
# "Predicted label: 7"

# Yeah! The model learned what the number 7 looks like, and it corredctly predicted the the image was indeed the number 7


In [ ]:
#######################################################################################################
# Recurrent Neural Network (RNN):

# A Recurrent Neural Network (RNN), is a type of artificial neural network designed to handle sequential data. Unlike traditional 
# feedforward neural networks, where information flows in one direction (from input to output), RNNs have connections that form 
# directed cycles, allowing them to exhibit temporal dynamic behavior.

# Here are the key components of an RNN:

# Recurrent Connections: The defining feature of RNNs is the presence of recurrent connections that allow information to persist 
# over time. This means that the output of a neuron at a given time step serves as an input to the same neuron at the next time step.

# Hidden State: RNNs maintain a hidden state that represents a memory of the previous time steps. This hidden state is updated at each 
# time step based on the current input and the previous hidden state.

# Variable Length Sequences: RNNs can process sequences of variable length, making them suitable for tasks involving sequences like 
# time series prediction, natural language processing, speech recognition, and more.

# Types of RNNs:
#  One-to-One: Traditional feedforward neural networks where there's no sequential processing involved.
#  One-to-Many: Takes a single input and generates a sequence of outputs (e.g., image captioning).
#  Many-to-One: Processes a sequence of inputs and produces a single output (e.g., sentiment analysis).
#  Many-to-Many (sequence-to-sequence): Takes a sequence of inputs and produces a sequence of outputs (e.g., machine translation).
#  Bidirectional: Processes the input sequence in both forward and backward directions, allowing the model to capture dependencies 
#  from both past and future contexts.

# Vanishing Gradient Problem: RNNs are susceptible to the vanishing gradient problem, where gradients become extremely small during 
# backpropagation, leading to difficulties in learning long-term dependencies. This limitation has led to the development of more 
# sophisticated RNN variants like Long Short-Term Memory (LSTM) networks and Gated Recurrent Units (GRUs), which are designed to 
# address this issue by introducing mechanisms to control the flow of information through the network.

# In the example below, we will do a simple language translation exercise, which falls into the category of Many-to-Many (sequence-to-sequence)


import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.models import Model

# Example data
input_texts = ['hello', 'nice to meet youy', 'goodbye']
target_texts = ['bonjour', 'ravi de vous rencontrer', 'au revoir']

# Tokenization (character level)
input_characters = sorted(set(''.join(input_texts)))
target_characters = sorted(set(''.join(target_texts)))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

# Add start and end tokens to target characters
start_token = '\t'
end_token = '\n'
target_characters = [start_token, end_token] + target_characters
num_decoder_tokens = len(target_characters)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

# Prepare data for training
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0


# Define the Seq2Seq model architecture
latent_dim = 256

encoder_inputs = tf.keras.Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = tf.keras.Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile and train the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=1, epochs=50)

# Define encoder and decoder models for inference
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = tf.keras.Input(shape=(latent_dim,))
decoder_state_input_c = tf.keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

# Function to decode sequence
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.0  # Start token

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length or find stop token
        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1) and states
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]

    return decoded_sentence

# Test the model on new input sequences
for seq_index in range(len(input_texts)):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    

# Final Result:   
    
# Input sentence: hello
# Decoded sentence: onjoerrrrrrrrrrrrrrrrrrr

# Input sentence: nice to meet youy
# Decoded sentence: avi         rrrrrrrrrrrr

# Input sentence: goodbye
# Decoded sentence: onjoerrrrrrrrrrrrrrrrrrr


In [ ]:
#######################################################################################################
# Convolutional Neural Network (CNN):

# A Convolutional Neural Network (CNN) is a type of deep neural network that is primarily used for tasks involving image 
# processing and computer vision. CNNs are particularly effective for tasks such as image classification, object detection, 
# image segmentation, and image generation. Here are some common applications of Convolutional Neural Networks:

# Image Classification: CNNs are widely used for image classification tasks, where the goal is to classify input images into 
# predefined categories or classes. For example, classifying images of animals into different species or identifying different 
# objects in a scene.

# Object Detection: CNNs can be used for object detection tasks, where the goal is to detect and localize objects within an 
# image and classify them into predefined categories. This is commonly used in applications such as autonomous vehicles, 
# surveillance systems, and medical imaging.

# Image Segmentation: CNNs can segment images into different regions or objects, where each pixel in the image is assigned a 
# class label. This is useful for tasks such as medical image analysis, semantic segmentation, and scene understanding.

# Feature Extraction: CNNs are often used as feature extractors in conjunction with other machine learning algorithms. By 
# leveraging the learned representations from CNNs, features can be extracted from input images and used as input to classifiers 
# or other models for various tasks.

# Image Generation: CNNs can also be used for generative tasks, where the goal is to generate new images that are similar to a 
# given dataset. This is commonly used in applications such as image synthesis, style transfer, and image enhancement.
    
# In the example below, we will focus on image classification, specifically we will loop through several images in a folder, 
# which has several sub-folders, each sub-folder having seperate classes of images, then train the model, and finally load one 
# image and get the model to classify that one image.


import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models


path_to_folder_with_subfolders = 'C:\\Users\\CNN\\Several Images\\'
 
# Define parameters
image_size = (128, 128)
batch_size = 32
num_classes = len(os.listdir(path_to_folder_with_subfolders))

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Splitting the data into training and validation sets
)

train_generator = train_datagen.flow_from_directory(
    path_to_folder_with_subfolders,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Using the training subset of the data
)

validation_generator = train_datagen.flow_from_directory(
    path_to_folder_with_subfolders,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Using the validation subset of the data
)

# Define the CNN architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=5, validation_data=validation_generator)

# Save the trained model
# model.save('image_classification_model.h5')

# Load the trained model
# loaded_model = tf.keras.models.load_model('image_classification_model.h5')

# Load and preprocess the single image for classification
img_path = 'C:\\Users\\CNN\\image.jpg'
img = tf.keras.preprocessing.image.load_img(img_path, target_size=image_size)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

# Make predictions
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)

# Get class labels
class_labels = list(train_generator.class_indices.keys())

# Print the predicted class
print("Predicted class:", class_labels[predicted_class])

# Final Result:  
# "Predicted class: mouse"

# Yeah! The model looped through several sub-folders, each of which contained images of 'cat', 'dog', 'squirrel', 'giraffe' and 'mouse', 
# and it it corredctly predicted the the 'image.jpg' was indeed an image of a mouse!


In [ ]:
#######################################################################################################
# Gated Recurrent Unit (GRU):

# Gated Recurrent Units (GRUs) are a type of recurrent neural network (RNN) architecture that is commonly used for 
# various sequence modeling tasks. Here are some common types of analysis and tasks that can be performed using GRUs:

#  Sequence Prediction:
#  GRUs can be used for sequence prediction tasks, where the goal is to predict the next element in a sequence based on the previous elements. 
#  Examples include time series forecasting, stock price prediction, weather forecasting, and natural language processing tasks such as 
#  next-word prediction.

#  Sequence Classification:
#  GRUs can be applied to sequence classification tasks, where the goal is to classify an entire sequence into one or 
#  more categories. Examples include sentiment analysis of text data, activity recognition from sensor data, and speech recognition.

#  Sequence Generation:
#  GRUs can generate sequences of data, such as text, music, or images, based on learned patterns from training data.
#  Examples include text generation, music composition, and image captioning.

#  Sequence-to-Sequence Learning:
#  GRUs can be used for sequence-to-sequence learning tasks, where an input sequence is mapped to an output sequence.
#  Examples include machine translation, question answering, and chatbot systems.

#  Temporal Pattern Recognition:
#  GRUs can recognize temporal patterns in sequential data and learn long-term dependencies between elements in the sequence.
#  Examples include gesture recognition in videos, anomaly detection in time-series data, and event detection in sensor data.

#  Feature Learning:
#  GRUs can automatically learn useful representations or features from sequential data, which can then be used as input for 
#  downstream machine learning tasks.
#  Examples include feature extraction from time-series data, speech recognition features, and text embeddings for natural 
#  language processing tasks.

#  Time-Series Analysis:
#  GRUs can analyze and model time-series data to detect patterns, trends, and anomalies.
#  Examples include financial market forecasting, medical signal analysis, and industrial process monitoring.


# In the example below, we are using a Gated Recurrent Unit (GRU) for next word prediction and for generating text embeddings 
# for natural language processing (NLP) tasks.

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

# Sample text data
text_data = [
    "The quick brown fox jumps over the lazy dog",
    "The cat sat on the mat",
    "We need to do grocery shopping",
    "This is a very nice hotel",
    "I love machine learning"
]

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

# Create input sequences and labels
input_sequences = []
for line in text_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences for equal length
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create predictors and labels
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

# One-hot encode the labels
label = tf.keras.utils.to_categorical(label, num_classes=total_words)

# Build the GRU model
model = Sequential([
    Embedding(total_words, 100, input_length=max_sequence_len - 1),
    GRU(150),
    Dense(total_words, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(predictors, label, epochs=100, verbose=1)


# Function to generate next word predictions
def predict_next_word(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list)[0]
        predicted_index = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Generate next word predictions
print(predict_next_word("We need", 3))


# Final Result: 
# "We need to do grocery"


In [ ]:
#######################################################################################################
# Generative Adversarial Network (GAN):

# Generative Adversarial Networks (GANs) are a class of deep learning models that are primarily used for generating new 
# data samples that are similar to a given dataset. GANs consist of two neural networks: a generator and a discriminator, 
# which are trained simultaneously in a competitive manner. While the primary application of GANs is data generation, they 
# can also be used for various other types of analysis and tasks. Here are some common types of analysis that can be performed 
# using Generative Adversarial Networks (GANs):

#  Data Generation:
#  GANs are most commonly used for generating new data samples that are similar to a given dataset. This can include generating 
#  realistic images, videos, audio, text, and other types of data.
#  Example applications include generating synthetic images of faces, bedrooms, or other objects; generating realistic artwork 
#  or music; and generating realistic text or speech.

#  Data Augmentation:
#  GANs can be used to augment existing datasets by generating additional synthetic samples. This can help in increasing the 
#  diversity of the dataset and improving the performance of machine learning models trained on limited data.
#  Example applications include generating additional images for training computer vision models and generating additional text 
#  samples for training natural language processing models.

#  Style Transfer:
#  GANs can be used for style transfer, where the style of one image or dataset is transferred to another image or dataset. This 
#  can be used for artistic purposes or for transferring the style of one domain to another.
#  Example applications include transferring the style of one painting to another painting, transferring the style of one photograph 
#  to another photograph, and transferring the style of one type of music to another type of music.

#  Anomaly Detection:
#  GANs can be used for anomaly detection by training the discriminator on normal data samples and then using the generator to generate 
#  synthetic samples. Anomalies can be detected by measuring the difference between the generated samples and the real samples.
#  Example applications include detecting anomalies in medical images, financial transactions, and network traffic.

#  Domain Adaptation:
#  GANs can be used for domain adaptation, where the goal is to transfer knowledge learned from one domain to another domain with 
#  different characteristics. This can help in improving the performance of machine learning models when there is a mismatch between 
#  the training and test data distributions.
#  Example applications include adapting a model trained on synthetic data to real-world data, adapting a model trained on one type of 
#  sensor data to another type of sensor data, and adapting a model trained on one language to another language.

#  Image-to-Image Translation:
#  GANs can be used for image-to-image translation, where the goal is to translate an image from one domain to another domain while 
#  preserving important features such as content and style.
#  Example applications include translating images from one artistic style to another, translating images from one domain to another 
#  (e.g., day to night, black and white to color), and translating images between different modalities (e.g., sketch to photograph, 
#  satellite image to map).
        


import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Conv2DTranspose
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model



# Function to load and preprocess images from a folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = load_img(os.path.join(folder, filename), target_size=(64, 64))
        img_array = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
        images.append(img_array)
    return np.array(images)

# Load images from a folder
image_folder = 'C:\\Users\\ryan_\\Desktop\\Briefcase\\PDFs\\1-ALL PYTHON & R CODE SAMPLES\\A - GITHUB\\Neural Networks - FNN vs CNN vs RNN vs GRU vs GAN\\CNN\\Several Images\\dog\\'
X_train = load_images_from_folder(image_folder)

# Generator model
generator = Sequential([
    Dense(8 * 8 * 256, input_dim=100, activation='relu'),
    Reshape((8, 8, 256)),
    Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', activation='relu'),
    Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', activation='relu'),
    Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', activation='sigmoid')
])

# Define a separate model for training the generator
generator_input = Input(shape=(100,))
generated_image = generator(generator_input)

generator_model = Model(generator_input, generated_image)

# Compile the generator model
generator_model.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')

# Generate noise as input to the generator
noise = np.random.normal(0, 1, (len(X_train), 100))  # Generating noise samples for each image in the dataset

epochs = 1000
batch_size = 1000

# Train the generator on noise
generator_model.fit(noise, X_train, epochs=epochs, batch_size=batch_size)

# Generate fake images after training
fake_images = generator.predict(noise)

# Display generated images
plt.figure(figsize=(8, 8))
for i in range(min(16, len(fake_images))):
    plt.subplot(4, 4, i + 1)
    plt.imshow(fake_images[i])
    plt.axis('off')
plt.tight_layout()
plt.show()


# Final Result:

# The GAN is essentially learning to mimic the distribution of the images it's trained on. If you only have three 
# images of dogs in your dataset, the GAN will learn to generate images that resemble those three images but won't 
# be able to create entirely new variations of dogs that it hasn't seen before.

# To generate entirely new images of dogs that don't exist in your original dataset, you'd typically need a much 
# larger and diverse dataset. This would enable the GAN to learn a richer representation of the characteristics of 
# dogs and be able to generate novel variations.


In [ ]:

# END!!!
